In [1]:
from utils.config_helper import create_yaml
import os
dir = '/braintree/home/aliya277/inventory/'
array_meta_path  = '/braintree/data2/active/users/sgouldin/array-metadata'
for files in os.listdir(dir):
    num_files = 0
    path = os.path.join(dir, files)
    try: 
        num_files = len(os.listdir(os.path.join(path,'SpikeTimes')))
    except: pass

    if num_files == 192: 
        array_metadata = os.path.join(array_meta_path, '021023_pico_mapping_noCIT_adapter_version.json')
        adapter_info_avail = True
    elif num_files == 288: 
        array_metadata = os.path.join(array_meta_path,'pico_firstmapping_Lhem_2023.json')
        adapter_info_avail = False
        
        print(path)
    create_yaml(path, array_metadata, adapter_info_avail)

    # except: print(f'Check: {files}')

/braintree/home/aliya277/inventory/norm_FOSS.sub_pico.20220901_144739.proc
/braintree/home/aliya277/inventory/norm_FOSS.sub_pico.20220929_170635.proc
/braintree/home/aliya277/inventory/norm_FOSS.sub_pico.20220930_151743.proc
/braintree/home/aliya277/inventory/norm_FOSS.sub_pico.20220615_113442.proc
/braintree/home/aliya277/inventory/norm_FOSS.sub_pico.20220902_145351.proc
/braintree/home/aliya277/inventory/norm_FOSS.sub_pico.20220907_142157.proc
/braintree/home/aliya277/inventory/exp_ko_context_size.sub_pico.20220930_153358.proc
/braintree/home/aliya277/inventory/norm_FOSS.sub_pico.20220616_111545.proc
/braintree/home/aliya277/inventory/exp_MURI1320.sub_pico.20220615_115139.proc


In [18]:
import io, os, pynwb
from pynwb import NWBHDF5IO
from nwbwidgets import nwb2widget
############### Check if All Files are Written and can be Opened ##############
###############################################################################

inventory   = '/braintree/home/aliya277/inventory/'
all_files = os.listdir(inventory)
for folder in all_files[:24]:
        path = os.path.join(inventory, folder)
        try:
                io = NWBHDF5IO(os.path.join(path, f"{folder}.nwb"), "r") 
                nwbfile = io.read()
                io.close()
        except: print(f'This File can not be opened: {folder}')


In [19]:
############### Validate All Files Using pwnyb, nwbinspectors #################
###############################################################################

import io, os, pynwb
from pynwb import validate
from nwbinspector import inspect_nwbfile

inventory   = '/braintree/home/aliya277/inventory/'
all_nwb_paths = []
all_files = os.listdir(inventory)
for folder in all_files[:24]:
    path = os.path.join(inventory, folder, f'{folder}.nwb')
    all_nwb_paths.append(path)
    


pynwb_validation = validate(paths = all_nwb_paths)

nwbinspector_validation = []
for path in all_nwb_paths:
    results = list(inspect_nwbfile(nwbfile_path=path))
    nwbinspector_validation.append(results)

In [29]:
from dandi import validate 

dandi = validate.validate(all_nwb_paths[0])
print(dandi)

<generator object validate at 0x7f69be3c3cd0>


In [20]:
nwbinspector_validation

[[],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 []]

In [21]:
pynwb_validation

([], 0)

In [11]:
import io, os, pynwb
from pynwb import NWBHDF5IO
from nwbwidgets import nwb2widget

inventory   = '/braintree/home/aliya277/inventory/'
all_files = os.listdir(inventory)
folder = all_files[0]
path = os.path.join(inventory, folder, f'{folder}.nwb')
io = NWBHDF5IO(path, "r") 
nwbfile = io.read()
# io.close()
nwb2widget(nwbfile)

In [14]:
nwbfile.subject.sex = 'M'

AttributeError: can't set attribute 'sex' -- already set

In [6]:
import io, os, pynwb
inventory   = '/braintree/home/aliya277/inventory/'
all_nwb_paths = []
all_files = os.listdir(inventory)
for folder in all_files:
    path = os.path.join(inventory, folder, f'{folder}.nwb')
    all_nwb_paths.append(path)
all_files

['exp_Alireza_paradigm1.sub_pico.20230616_103724.proc',
 'exp_domain_transfer_2023.sub_pico.20230216_150919.proc',
 'exp_domain_transfer_2023.sub_pico.20230223_134049.proc',
 'exp_facesMSFDE.sub_pico.20230327_134955.proc',
 'exp_gratingsAdap_s3.sub_pico.20230801_163355.proc',
 'exp_gratingsAdap_s5.sub_pico.20230801_144134.proc',
 'exp_HVM_var6.sub_pico.20230221_142542.proc',
 'exp_HVM_var6.sub_pico.20230223_150327.proc',
 'exp_HVM-var6-subset.sub_pico.20230303_180103.proc',
 'exp_images_in_context3.sub_pico.20230531_135649.proc',
 'exp_images_in_context.sub_pico.20230324_121431.proc',
 'exp_Mayo_day_1.sub_pico.20230814_124244.proc',
 'exp_Mayo_day_1.sub_pico.20230814_143555.proc',
 'exp_Mayo_day_2.sub_pico.20230815_111611.proc',
 'exp_Mayo_day_5.sub_pico.20230818_121533.proc',
 'exp_Mayo_ONR.sub_pico.20230804_154202.proc',
 'exp_Mayo_ONR.sub_pico.20230808_143723.proc',
 'exp_moca.sub_pico.20230719_123506.proc',
 'exp_MURI1320-2023.sub_pico.20230131_183129.proc',
 'exp_MURI1320-2023.sub

In [7]:
from pynwb import validate

validate(paths = all_nwb_paths[0:5])

([], 0)

In [17]:
from nwbinspector import inspect_nwbfile

for path in all_nwb_paths[0:10]:
    results = list(inspect_nwbfile(nwbfile_path=path))
    print(results)

[InspectorMessage(
    message='Description is missing.',
    importance=<Importance.BEST_PRACTICE_SUGGESTION: 0>,
    severity=<Severity.LOW: 1>,
    check_function_name='check_description',
    object_type='Subject',
    object_name='subject',
    location='/general/subject',
    file_path='/braintree/home/aliya277/inventory/exp_Alireza_paradigm1.sub_pico.20230616_103724.proc/exp_Alireza_paradigm1.sub_pico.20230616_103724.proc.nwb'
), InspectorMessage(
    message="Subject.sex should be one of: 'M' (male), 'F' (female), 'O' (other), or 'U' (unknown).",
    importance=<Importance.BEST_PRACTICE_SUGGESTION: 0>,
    severity=<Severity.LOW: 1>,
    check_function_name='check_subject_sex',
    object_type='Subject',
    object_name='subject',
    location='/general/subject',
    file_path='/braintree/home/aliya277/inventory/exp_Alireza_paradigm1.sub_pico.20230616_103724.proc/exp_Alireza_paradigm1.sub_pico.20230616_103724.proc.nwb'
)]
[InspectorMessage(
    message='Description is missing.'